In [1]:


import numpy as np
import time
import timeit

SIZE = [500*500]
RUNS = 10_000


def mul():
    return ARR*ARR

ARR = np.random.choice([5,0], size=SIZE, p=[0.5,0.5])
print(timeit.timeit(stmt=mul, number=RUNS))

ARR = np.random.choice([5,0], size=SIZE, p=[0.1,0.9])
print(timeit.timeit(stmt=mul, number=RUNS))

ARR = np.random.uniform(1, 10, size=SIZE)
print(timeit.timeit(stmt=mul, number=RUNS))


1.2234761960000355
1.3051025839999966
1.1912348160000192


In [16]:
import torch
from torch import nn
import torch.nn.utils.prune as prune
import torch.nn.functional as F


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        # 1 input image channel, 6 output channels, 3x3 square conv kernel
        self.conv1 = nn.Conv2d(1, 6, 3)
        self.conv2 = nn.Conv2d(6, 16, 3)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)  # 5x5 image dimension
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, int(x.nelement() / x.shape[0]))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

model = LeNet().to(device=device)


module = model.conv1
print(list(module.named_parameters()))

[('weight', Parameter containing:
tensor([[[[ 0.0696,  0.2067, -0.2556],
          [ 0.1500,  0.2942,  0.2083],
          [-0.2191, -0.0414, -0.2277]]],


        [[[ 0.1929, -0.3156,  0.1858],
          [ 0.0647, -0.2856, -0.0058],
          [-0.0423,  0.0845,  0.3201]]],


        [[[-0.2524, -0.0075, -0.2302],
          [-0.0682, -0.1007, -0.3304],
          [-0.2870,  0.3072, -0.0886]]],


        [[[ 0.0130, -0.0439, -0.1026],
          [ 0.1086,  0.0526,  0.0934],
          [ 0.0481,  0.1797, -0.1982]]],


        [[[-0.2066, -0.0199, -0.1472],
          [ 0.0007,  0.2304, -0.2892],
          [-0.1760,  0.1045, -0.2712]]],


        [[[ 0.2044,  0.0025, -0.2743],
          [ 0.1469, -0.3179, -0.1738],
          [ 0.1075,  0.2918,  0.2553]]]], requires_grad=True)), ('bias', Parameter containing:
tensor([ 0.1417,  0.2496,  0.1655,  0.2093, -0.0947,  0.1983],
       requires_grad=True))]


In [18]:
prune.random_unstructured(module, name="weight", amount=0.3)
print(list(module.named_parameters()))

[('bias', Parameter containing:
tensor([ 0.1417,  0.2496,  0.1655,  0.2093, -0.0947,  0.1983],
       requires_grad=True)), ('weight_orig', Parameter containing:
tensor([[[[ 0.0696,  0.2067, -0.2556],
          [ 0.1500,  0.2942,  0.2083],
          [-0.2191, -0.0414, -0.2277]]],


        [[[ 0.1929, -0.3156,  0.1858],
          [ 0.0647, -0.2856, -0.0058],
          [-0.0423,  0.0845,  0.3201]]],


        [[[-0.2524, -0.0075, -0.2302],
          [-0.0682, -0.1007, -0.3304],
          [-0.2870,  0.3072, -0.0886]]],


        [[[ 0.0130, -0.0439, -0.1026],
          [ 0.1086,  0.0526,  0.0934],
          [ 0.0481,  0.1797, -0.1982]]],


        [[[-0.2066, -0.0199, -0.1472],
          [ 0.0007,  0.2304, -0.2892],
          [-0.1760,  0.1045, -0.2712]]],


        [[[ 0.2044,  0.0025, -0.2743],
          [ 0.1469, -0.3179, -0.1738],
          [ 0.1075,  0.2918,  0.2553]]]], requires_grad=True))]


In [21]:
print(list(module.named_buffers()))

print(module.weight)
print(module._forward_pre_hooks)

[('weight_mask', tensor([[[[1., 0., 1.],
          [0., 0., 1.],
          [1., 1., 0.]]],


        [[[1., 0., 1.],
          [0., 0., 0.],
          [0., 1., 1.]]],


        [[[0., 1., 0.],
          [0., 0., 1.],
          [0., 0., 1.]]],


        [[[0., 1., 1.],
          [1., 1., 1.],
          [1., 1., 0.]]],


        [[[1., 1., 0.],
          [0., 1., 0.],
          [0., 0., 1.]]],


        [[[1., 0., 0.],
          [1., 0., 0.],
          [1., 1., 0.]]]]))]
tensor([[[[ 0.0696,  0.0000, -0.2556],
          [ 0.0000,  0.0000,  0.2083],
          [-0.2191, -0.0414, -0.0000]]],


        [[[ 0.1929, -0.0000,  0.1858],
          [ 0.0000, -0.0000, -0.0000],
          [-0.0000,  0.0845,  0.3201]]],


        [[[-0.0000, -0.0075, -0.0000],
          [-0.0000, -0.0000, -0.3304],
          [-0.0000,  0.0000, -0.0886]]],


        [[[ 0.0000, -0.0439, -0.1026],
          [ 0.1086,  0.0526,  0.0934],
          [ 0.0481,  0.1797, -0.0000]]],


        [[[-0.2066, -0.0199, -0.0000],
   